# Calibration
---

This notebook demonstates calibration of a 4D-STEM dataset for the purposes of strain mapping.  The following calibration measurements are performed:

- center position (i.e. origin of coordinates in diffraction space)
- elliptical distortion of diffraction space
- rotational misalignment of real and diffraction space directions
- detector (diffraction space) pixel size
- beam convergence angle


## Data
This is a simulated 4D-STEM dataset.  Simulations were performed by Colin Ophus, have DOI number 10.5281/zenodo.3592520, and can be [downloaded here](https://drive.google.com/file/d/1QiH7phMR0AaMkYoio3uhgTTQMOHG4l6b/view?usp=sharing).  
You should then set the `filepath` variable in the cell below.


### Versioning

Last updated on 2021-04-23 with py4DSTEM v.0.12.0.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import py4DSTEM
from os import path 

In [ ]:
filepath = "/media/AuxDriveB/Data/4DSTEM_SampleData/py4DSTEM_sample_data/calibrationData_simulatedAuNanoplatelet_binned.h5"

## Load data

In [ ]:
py4DSTEM.io.read(filepath)

### What this data is

The data here is meant to represent everything needed to perform the calibrations required for strain mapping.  The data in which strain is to be measuremed and analyzed is not required at this point.  The idea is that for a single day of experiments and a single set of experimental conditions, these calibrations need only be performed once.  The output file of this notebook can then be used to calibrate and analyze many 4D-STEM datasets.

#### `datacube_cal` (polyAu_4DSTEM)

This is a 4D-STEM scan of a calibartion sample - ideally, this should be a sample with known structure / lattice parameters, and with many crystal orientations represented.  Here we use a polydisperse distribution of gold nanoparticles.  The collection of many rotation angles of a crystal of known lattice structure is useful for calibration of the diffraction space pixel size, as well as the elliptical distortions.

#### `probe_template`

py4DSTEM detects Bragg scattering using template matching - to use these methods, an image of the probe template should be collected. Bragg disk detection is used here for careful elliptical distortion measurement.

#### `defocused_probe` and `datacube_rotation` (simulation_4DSTEM)

The real and diffraction planes may, in general, have some rotational misalignment.  Here we determine this misalignment by measuring the rotation between two images of the same sample - one in the diffraction plane, and one in the real plane.  A diffraction plane image can be obtained by defocusing the beam will produce a shadow image of the sample inside the CBED pattern.  A real plane image can be obtained using any STEM imaging modality.  Here we used an image of the defocused probe and a virtual image generated from a 4D-STEM scan, both obtained with the beam incident on the same sample, to perform this calibration.

In [ ]:
datacube_cal = py4DSTEM.io.read(filepath,data_id='polyAu_4DSTEM')
probe_template = py4DSTEM.io.read(filepath,data_id='probe_template').data
defocused_probe = py4DSTEM.io.read(filepath,data_id='defocused_probe').data
datacube_rotation = py4DSTEM.io.read(filepath,data_id='simulation_4DSTEM')

#### coordinates

The calibrations performed here are stored in a Coordinates instance, one of the datastructures py4DSTEM knows how to read/write.

In [ ]:
coordinates = py4DSTEM.io.datastructure.Coordinates(datacube_cal.R_Nx,datacube_cal.R_Ny,
                                                   datacube_cal.Q_Nx,datacube_cal.Q_Nx,
                                                   name='coordinates_calibrationdata')

## Examine the data

In [ ]:
# Examine the 4D calibration dataset
dp_max_cal = np.max(datacube_cal.data,axis=(0,1))
py4DSTEM.visualize.show(dp_max_cal,scaling='log')

In [ ]:
# Bright-field image
qx0,qy0 = 64,64
qR = 12

py4DSTEM.visualize.show_circles(dp_max_cal,center=(qx0,qy0),R=qR,alpha=0.25,scaling='log')
BF_cal = py4DSTEM.process.virtualimage.get_virtualimage_circ(datacube_cal,qx0,qy0,qR)
py4DSTEM.visualize.show(BF_cal)

In [ ]:
# Probe template
py4DSTEM.visualize.show(probe_template,scaling='log')

In [ ]:
# defocused probe image
py4DSTEM.visualize.show(defocused_probe)

In [ ]:
# Examine the 4D dataset of interest
dp_max_rotation = np.max(datacube_rotation.data,axis=(0,1))
py4DSTEM.visualize.show(dp_max_rotation,scaling='log')

In [ ]:
# Get a virtual bright-field image
qx0,qy0 = 63.5,63.45
qR = 12

py4DSTEM.visualize.show(dp_max_rotation,scaling='log',
                        circle={'center':(qx0,qy0),'R':qR,'alpha':.25,'fill':True})
BF_rotation = py4DSTEM.process.virtualimage.get_virtualimage_circ(datacube_rotation,qx0,qy0,qR)
py4DSTEM.visualize.show(BF_rotation)

## Prepare the probe template

Here we
- measure the center position and radius of the probe image
- generate a probe kernel for the template-matching disk detection step

Creating a good probe kernel is *essential* for the disk detection algorithm to work well - tuning the parameters you pass to `find_Bragg_disks` won't do you a lick of good if your kernel is no good.  More discussion of what makes a good probe template, and how to generate one, coming soon to a demo notebook near you.

In [ ]:
# Get the probe radius
r,qx0,qy0 = py4DSTEM.process.calibration.get_probe_size(probe_template)
r_trench = r+0.6
py4DSTEM.visualize.show_circles(probe_template,(qx0,qy0),r_trench,scaling='log')

In [ ]:
# Get the probe kernel
probe_kernel = py4DSTEM.process.diskdetection.get_probe_kernel_logistictrench(
                                        probe_template,r_trench,trenchwidth=3,blurwidth=1)
py4DSTEM.visualize.show_kernel(probe_kernel,R=30,L=64,W=3)

## Find the origin

Here we:
- measure the position of the origin 
- mask any outlier positions 
- fit a plane to those positions
- set the fit plane as the origin of coordinates at each scan position

In [ ]:
# Find CoM of center disk
qx0_meas,qy0_meas = py4DSTEM.process.calibration.get_origin(datacube=datacube_cal)
py4DSTEM.visualize.show_image_grid(get_ar=lambda i:[qx0_meas,qy0_meas][i],H=1,W=2,cmap='RdBu')

In [ ]:
# Set a mask for outliers
mask,scores,cutoff = py4DSTEM.process.calibration.find_outlier_shifts(qx0_meas,qy0_meas,
                                                n_sigma=5,edge_boundary=0)
py4DSTEM.visualize.show_hist(scores,vlines=cutoff)
py4DSTEM.visualize.show_image_grid(get_ar=lambda i:[qx0_meas,qy0_meas][i],
                                             H=1,W=2,cmap="RdBu",mask=mask==False)

In [ ]:
# Fit a plane
qx0_fit,qy0_fit,qx0_residuals,qy0_residuals = \
            py4DSTEM.process.calibration.fit_origin(qx0_meas,qy0_meas,mask=mask,fitfunction='parabola')
py4DSTEM.visualize.show_image_grid(lambda i:[qx0_meas,qx0_fit,qx0_residuals,
                                             qy0_meas,qy0_fit,qy0_residuals][i],
                                   H=2,W=3,cmap='RdBu')

In [ ]:
# Store the origin position
coordinates.set_origin(qx0_fit,qy0_fit)

## Find bragg disk positions

Here, we
- select a few diffraction patterns to use as examples
- tune the disk fitting parameters
- perform the disk fitting
- center the detected disk positions
- compute the bragg vector map (a 2D binned histogram of bragg peak positions and intensities)

In [ ]:
# Select a few DPs on which to test disk detection parameters
rxs_cal = 20,50,52
rys_cal = 5,31,78
colors = ['r','b','g']

py4DSTEM.visualize.show_points(BF_cal,x=rxs_cal,y=rys_cal,pointcolor=colors,figsize=(8,8))
py4DSTEM.visualize.show_image_grid(get_ar=lambda i:datacube_cal.data[rxs_cal[i],rys_cal[i],:,:],
                                   H=1,W=3,get_bordercolor=lambda i:colors[i],scaling='log')

In [ ]:
# Tune disk detection parameters on selected DPs
corrPower=1
sigma=2
edgeBoundary=4
minRelativeIntensity=0.05
relativeToPeak=0
minPeakSpacing=4
maxNumPeaks=80
subpixel='multicorr'
upsample_factor=16

selected_peaks = py4DSTEM.process.diskdetection.find_Bragg_disks_selected(
                        datacube=datacube_cal,
                        probe=probe_kernel,
                        Rx=rxs_cal,
                        Ry=rys_cal,
                        corrPower=corrPower,
                        sigma=sigma,
                        edgeBoundary=edgeBoundary,
                        minRelativeIntensity=minRelativeIntensity,
                        relativeToPeak=relativeToPeak,
                        minPeakSpacing=minPeakSpacing,
                        maxNumPeaks=maxNumPeaks,
                        subpixel=subpixel,
                        upsample_factor=upsample_factor
)

py4DSTEM.visualize.show_points(BF_cal,x=rxs_cal,y=rys_cal,pointcolor=colors,figsize=(8,8))
py4DSTEM.visualize.show_image_grid(get_ar=lambda i:datacube_cal.data[rxs_cal[i],rys_cal[i],:,:],H=1,W=3,
                                   get_bordercolor=lambda i:colors[i],
                                   get_x=lambda i:selected_peaks[i].data['qx'],
                                   get_y=lambda i:selected_peaks[i].data['qy'],
                                   get_pointcolors=lambda i:colors[i],scaling='log')

In [ ]:
# Get all disks
braggpeaks_raw = py4DSTEM.process.diskdetection.find_Bragg_disks(
                                datacube=datacube_cal,
                                probe=probe_kernel,
                                corrPower=corrPower,
                                sigma=sigma,
                                edgeBoundary=edgeBoundary,
                                minRelativeIntensity=minRelativeIntensity,
                                relativeToPeak=relativeToPeak,
                                minPeakSpacing=minPeakSpacing,
                                maxNumPeaks=maxNumPeaks,
                                subpixel=subpixel,
                                upsample_factor=upsample_factor,
                                name='braggpeaks_cal_raw'
)

In [ ]:
# Center the disk positions about the origin
braggpeaks_centered = py4DSTEM.process.calibration.center_braggpeaks(braggpeaks_raw,coords=coordinates)

In [ ]:
# Compute the Bragg vector map
bvm_cal = py4DSTEM.process.diskdetection.get_bvm(braggpeaks_centered,datacube_cal.Q_Nx,datacube_cal.Q_Ny)
py4DSTEM.visualize.show(bvm_cal,cmap='inferno',scaling='power',power=0.5,clipvals='manual',min=0,max=50)

## Elliptical distortion calibration

Here we
- select an annular fitting region
- fit a 2D elliptical curve to this region of the BVM
- save the elliptical distortions to Coordinates
- correct the bragg disk positions by stretching along the semiminor axis until it matches the semimajor axis length
- check that the elliptical distortions have been removed from the corrected disk positions

In [ ]:
# Select fitting region
qmin,qmax = 31,39
py4DSTEM.visualize.show(bvm_cal,cmap='gray',scaling='log',clipvals='manual',min=0,max=15,
                        annulus={'center':(datacube_cal.Q_Nx/2.,datacube_cal.Q_Ny/2.),
                                 'Ri':qmin,'Ro':qmax,'fill':True,'color':'y','alpha':0.2})

In [ ]:
# Fit the elliptical distortions
qx0,qy0,a,e,theta = py4DSTEM.process.calibration.fit_ellipse_1d(
                        bvm_cal,datacube_cal.Q_Nx/2.,datacube_cal.Q_Ny/2.,qmin,qmax)
py4DSTEM.visualize.show_elliptical_fit(bvm_cal,
                       cmap='gray',scaling='log',clipvals='manual',min=0,max=15,
                       center=(qx0,qy0),Ri=qmin,Ro=qmax,a=a,e=e,theta=theta,fill=True)

In [ ]:
# Save to Coordinates
coordinates.set_ellipse(e,theta)

In [ ]:
# Confirm that elliptical distortions have been removed

# Correct bragg peak positions, stretching the elliptical semiminor axis to match the semimajor axis length
braggpeaks_ellipsecorr = py4DSTEM.process.calibration.correct_braggpeak_elliptical_distortions(
                                            braggpeaks_centered,e,theta)

# Recompute the bvm
bvm_ellipsecorr = py4DSTEM.process.diskdetection.get_bragg_vector_map(
                            braggpeaks_ellipsecorr,datacube_cal.Q_Nx,datacube_cal.Q_Ny)

# Fit an ellipse to the elliptically corrected bvm
qx0_corr,qy0_corr,a_corr,e_corr,theta_corr = py4DSTEM.process.calibration.fit_ellipse_1d(bvm_ellipsecorr,qx0,qy0,qmin,qmax)
py4DSTEM.visualize.show_elliptical_fit(bvm_ellipsecorr,center=(qx0_corr,qy0_corr),Ri=qmin,Ro=qmax,a=a_corr,e=e_corr,theta=theta_corr,fill=True,
                                       cmap='magma',scaling='power',power=0.5,clipvals='std',min=0,max=5)

# Print the ratio of the semi-axes before and after correction
print("The ratio of the semiminor to semimajor axes was measured to be")
print("")
print("\t{:.2f}% in the original data and".format(100*e))
print("\t{:.2f}% in the corrected data.".format(100*e_corr))

## Pixel size calibration

In [ ]:
# Radial integration
ymax = 300000
dq=0.25             # binsize for the x-axis

q,I_radial = py4DSTEM.process.utils.radial_integral(
                        bvm_ellipsecorr,datacube_cal.Q_Nx/2,datacube_cal.Q_Ny/2,dr=dq)
py4DSTEM.visualize.show_qprofile(q=q,intensity=I_radial,ymax=ymax)

In [ ]:
# Fit a gaussian to find a peak location
qmin,qmax = 32.5,37
A,mu,sigma = py4DSTEM.process.fit.fit_1D_gaussian(q,I_radial,qmin,qmax)

fig,ax = py4DSTEM.visualize.show_qprofile(q=q,intensity=I_radial,ymax=ymax,
                                          returnfig=True)
ax.vlines((qmin,qmax),0,ax.get_ylim()[1],color='r')
ax.vlines(mu,0,ax.get_ylim()[1],color='g')
ax.plot(q,py4DSTEM.process.fit.gaussian(q,A,mu,sigma),color='r')
plt.show()

In [ ]:
# Get pixel calibration
# At time of writing, one peak with a known spacing
# must be manually identified and entered
d_spacing_nm = 0.1442                           # This is the Au 022 peak
inv_nm_per_pixel = 1./(d_spacing_nm * mu)
py4DSTEM.visualize.show_qprofile(q=q*inv_nm_per_pixel,intensity=I_radial,
                                 ymax=ymax,xlabel='q (1/nm)')

In [ ]:
# Demonstrate consistency with known Au spacings
spacings_nm = np.array([0.1177,0.123,0.1442,0.2039,0.2355])   # 222, 113, 022, 002, 111
spacings_inv_nm = 1./spacings_nm

fig,ax = py4DSTEM.visualize.show_qprofile(q=q*inv_nm_per_pixel,intensity=I_radial,
                                 ymax=ymax,xlabel='q (1/nm)',returnfig=True)
ax.vlines(spacings_inv_nm,0,ax.get_ylim()[1],color='r')
plt.show()

In [ ]:
# Store
coordinates.set_Q_pixel_size(inv_nm_per_pixel)
coordinates.set_Q_pixel_units(r'nm$^{-1}$')

## Rotational calibration

Here we
- display the shadow image (diffraction plane) and the matching 4D-STEM virtual image (real plane)
- specify a pair of identical fiducial points on both images
- compute the rotational misalignment of the real and diffraction planes
- store the misalignment in Coordinates

In [ ]:
# Show the shadow image
py4DSTEM.visualize.show(defocused_probe,figsize=(6,6))
py4DSTEM.visualize.show(BF_rotation,figsize=(6,6))

In [ ]:
# Pick two fiducial points, locate them on each image
cbed_p1 = (154,204)
cbed_p2 = (212,401)
stem_p1 = (59,16.5)
stem_p2 = (15,37)

fig,(ax1,ax2) = plt.subplots(1,2,figsize=(12,6))
ax1.matshow(defocused_probe,cmap='gray')
ax2.matshow(BF_rotation,cmap='gray')
ax1.plot((cbed_p1[1],cbed_p2[1]),(cbed_p1[0],cbed_p2[0]),color='y')
ax1.scatter((cbed_p1[1],cbed_p2[1]),(cbed_p1[0],cbed_p2[0]),color=('r','b'))
ax2.plot((stem_p1[1],stem_p2[1]),(stem_p1[0],stem_p2[0]),color='y')
ax2.scatter((stem_p1[1],stem_p2[1]),(stem_p1[0],stem_p2[0]),color=('r','b'))
ax1.grid(True)
ax2.grid(True)
plt.show()

In [ ]:
# Measure the rotational offset
stem_angle = np.angle(stem_p2[0]+1j*stem_p2[1] - stem_p1[0]-1j*stem_p1[1])%(2*np.pi)
cbed_angle = np.angle(cbed_p2[0]+1j*cbed_p2[1] - cbed_p1[0]-1j*cbed_p1[1])%(2*np.pi)
QR_rotation = stem_angle-cbed_angle
print("Real space is rotated {} degrees counterclockwise with respect to diffraction space.".format(np.degrees(QR_rotation)))

In [ ]:
coordinates.set_QR_rotation(QR_rotation)
coordinates.set_QR_flip(False)

## Save

Here we save everything we'll need for strain mapping in a new .h5 file.  We:
- set a filepath
- convert the probe template into a format py4DSTEM knows how to save
- save the Coordinates and probe template

In [ ]:
filepath_output = "/media/AuxDriveB/Data/4DSTEM_SampleData/py4DSTEM_sample_data/calibrationData_simulatedAuNanoplatelet_binned_processing.h5"

In [ ]:
# Generate DataObjects of the probe and BF image
probe = py4DSTEM.io.DiffractionSlice(data=np.dstack([probe_template,probe_kernel]),
                                     slicelabels=['probe_template','probe_kernel'],
                                     name='probe')

In [ ]:
# Save
py4DSTEM.io.save(filepath_output,
                 data=[coordinates,
                       probe],
                 overwrite=True)